# Model Training with TensorFlow

Train a model that predicts whether or not a patient has diabetes, based on medical features.

### 1. Import the required libraries and packages.

You can safely ignore the TensorFlow import warnings.
TensorFlow typically produces these warnings on CPU-only environments where the use of accelerators, such as GPUs or certain CPU instruction sets, is limited or not available.

In [1]:
from typing import List, Dict

import pandas as pd

import tensorflow as tf

import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

tf.config.set_visible_devices([], 'GPU')  # Hides all GPUs
print("GPU Disabled. Running on CPU.")

2025-03-07 08:11:55.785608: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-07 08:11:55.799085: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-07 08:11:55.803149: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-07 08:11:55.814625: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-07 08:11:56.786031: W tensorflow/compiler/tf2

GPU Disabled. Running on CPU.


### 2. Load the data into a Pandas dataframe.

In [2]:
data = pd.read_csv('./data/diabetes.csv')

Split the data into two data frames: features (`X`) and target variable (`y`).

In [3]:
X = data.drop('Outcome', axis=1)
y = data['Outcome']

Inspect the two dataframes.

In [4]:
X.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33


In [5]:
y.head()

0    1
1    0
2    1
3    0
4    1
Name: Outcome, dtype: int64

Divide the data into training and test data sets. 

The `train_test_split` method of Scikit-learn can split the data set into random train and test subsets.

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=0
)

print(f"Number of samples in training set: {X_train.shape[0]}")
print(f"Number of samples in test set: {X_test.shape[0]}")

Number of samples in training set: 614
Number of samples in test set: 154


### 4. Create and train the model.

Define a simple neural network model with the Keras API of TensorFlow.
The network must take eight input features and output two target values, corresponding to the two possible outcomes, diabetes or no diabetes.
The network also defines two internal layers, with 20 and 10 neurons respectively.

In [7]:
# Seed for reproducible results
tf.random.set_seed(10)
tf.keras.utils.set_random_seed(10)

model = tf.keras.Sequential([
    tf.keras.layers.Dense(20, activation='relu', input_shape=(8,)),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

/opt/app-root/lib64/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Compile the model and define the loss function, the optimizer, and the training epochs.

In [12]:
epochs = 50

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

Train the model.

In [13]:
epoch_time_start = int(time.time())
model.fit(X_train, y_train, epochs=epochs, verbose=0.5)
epoch_time_end = int(time.time())
print(" The Model takes " + str(epoch_time_end - epoch_time_start) + " secs")

Epoch 1/50
Epoch 2/50
Epoch 3/50
Epoch 4/50
Epoch 5/50
Epoch 6/50
Epoch 7/50
Epoch 8/50
Epoch 9/50
Epoch 10/50
Epoch 11/50
Epoch 12/50
Epoch 13/50
Epoch 14/50
Epoch 15/50
Epoch 16/50
Epoch 17/50
Epoch 18/50
Epoch 19/50
Epoch 20/50
Epoch 21/50
Epoch 22/50
Epoch 23/50
Epoch 24/50
Epoch 25/50
Epoch 26/50
Epoch 27/50
Epoch 28/50
Epoch 29/50
Epoch 30/50
Epoch 31/50
Epoch 32/50
Epoch 33/50
Epoch 34/50
Epoch 35/50
Epoch 36/50
Epoch 37/50
Epoch 38/50
Epoch 39/50
Epoch 40/50
Epoch 41/50
Epoch 42/50
Epoch 43/50
Epoch 44/50
Epoch 45/50
Epoch 46/50
Epoch 47/50
Epoch 48/50
Epoch 49/50
Epoch 50/50
 The Model takes 3 secs


### 5. Evaluate the model metrics.

After the model is trained, evaluate the model against the test set.

In [10]:
# Compute the predictions (y_predictions) given the test data (y_predicted)
y_predicted_probabilities = model.predict(X_test)
y_predicted = tf.argmax(y_predicted_probabilities, axis=1)

# Compare the predicted values for the test set (y_predicted)
# against the expected values (y_test)
print("Classification Report:")
print(classification_report(y_test, y_predicted))

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.82      0.83       107
           1       0.62      0.66      0.64        47

    accuracy                           0.77       154
   macro avg       0.73      0.74      0.74       154
weighted avg       0.78      0.77      0.77       154



The trained model has an accuracy value of aproximately 78%.

You can improve the score by retraining the model after more sophisticated data engineering or by tweaking the model hyper parameters.

### 6. Test the model with sample cases.
Test the model with data from two patients: one patient with diabetes and one patient without diabetes.

In [11]:
# Tuple for textual display of prediction
classes = ('No diabetes', 'Diabetes')


def predict(patients: List[Dict]):
    features_as_lists = [list(patient.values()) for patient in patients]
    inputs_array = np.array(features_as_lists)
    prediction_probabilities = model.predict(inputs_array, verbose=0)
    # argmax gets the index of the maximum value in an array
    predictions = [classes[np.argmax(p)] for p in prediction_probabilities]
    return predictions


diabetes_patient = {
    "Pregnancies": 6.0,
    "Glucose": 110.0,
    "BloodPressure": 65.0,
    "SkinThickness": 15.0,
    "Insulin": 1.0,
    "BMI": 45.7,
    "DiabetesPedigreeFunction": 0.627,
    "Age": 50
}

no_diabetes_patient = {
    "Pregnancies": 0,
    "Glucose": 88.0,
    "BloodPressure": 60.0,
    "SkinThickness": 35.0,
    "Insulin": 1.0,
    "BMI": 45.7,
    "DiabetesPedigreeFunction": 0.27,
    "Age": 20
}

predictions = predict([diabetes_patient, no_diabetes_patient])
print(predictions)

['Diabetes', 'No diabetes']
